In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.__version__
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator()

from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.python.keras.layers.pooling import GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mobile = tf.keras.applications.mobilenet_v2.MobileNetV2()

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13026815334732441687
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2256220980
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17307819998812677351
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [4]:
train_path = 'data2/train'
valid_path = 'data2/valid'
test_path = 'data2/test'

In [5]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=64)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=64)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=64, shuffle=False)

Found 28446 images belonging to 80 classes.
Found 4800 images belonging to 80 classes.
Found 3200 images belonging to 80 classes.


In [16]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model = MobileNetV2(weights='imagenet' , include_top = False , input_shape=IMG_SHAPE)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128,activation='relu')(x)
preds = Dense(80, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=preds)

In [7]:
for layers in model.layers[:-7]:
    layers.trainable = False

In [8]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=30,
            verbose=2
)

Epoch 1/30
445/445 - 306s - loss: 2.8541 - accuracy: 0.3338 - val_loss: 1.8295 - val_accuracy: 0.5156 - 306s/epoch - 687ms/step
Epoch 2/30
445/445 - 116s - loss: 1.4816 - accuracy: 0.6061 - val_loss: 1.4522 - val_accuracy: 0.5987 - 116s/epoch - 262ms/step
Epoch 3/30
445/445 - 117s - loss: 1.1932 - accuracy: 0.6757 - val_loss: 1.2922 - val_accuracy: 0.6348 - 117s/epoch - 264ms/step
Epoch 4/30
445/445 - 102s - loss: 1.0316 - accuracy: 0.7181 - val_loss: 1.2189 - val_accuracy: 0.6546 - 102s/epoch - 228ms/step
Epoch 5/30
445/445 - 99s - loss: 0.9193 - accuracy: 0.7479 - val_loss: 1.1647 - val_accuracy: 0.6731 - 99s/epoch - 224ms/step
Epoch 6/30
445/445 - 99s - loss: 0.8305 - accuracy: 0.7728 - val_loss: 1.1411 - val_accuracy: 0.6792 - 99s/epoch - 222ms/step
Epoch 7/30
445/445 - 94s - loss: 0.7518 - accuracy: 0.7956 - val_loss: 1.1023 - val_accuracy: 0.6950 - 94s/epoch - 211ms/step
Epoch 8/30
445/445 - 86s - loss: 0.6862 - accuracy: 0.8146 - val_loss: 1.0852 - val_accuracy: 0.7013 - 86s/epo

In [10]:
model.save('models/model_4.h5')